In [ ]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import *
spark = pyspark.sql.SparkSession.builder.appName("Product_Price_Tracking") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [ ]:
df_productsaug20 = spark.read.csv('hdfs:///delta_lake/raw/products_aug20.csv', header=True, inferSchema=True)
df_productsaug20.show()

In [ ]:
df_productsaug20.write.format("delta").save("hdfs:///delta_lake/products")

In [ ]:
spark.sql("CREATE TABLE products USING DELTA LOCATION 'hdfs:///delta_lake/products'")
spark.sql('SELECT * FROM products').show()

In [ ]:
deltaTable = DeltaTable.forPath(spark, "hdfs:///delta_lake/products").alias("products")
spark.table("products").show()

In [ ]:
deltaTable.update("ProductID = '200'", { "Price": "'48.00'" } )

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 1).load("hdfs:///delta_lake/products")
df.show()

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load("hdfs:///delta_lake/products")
df.show()

In [ ]:
deltaTable.delete("ProductID = 210") 

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 2).load("hdfs:///tmp/products")
df.show()

In [ ]:
df_productsaug21 = spark.read.csv('hdfs:///delta_lake/raw/products_aug21.csv', header=True, inferSchema=True)
df_productsaug21.show()

In [ ]:
deltaTable.alias("products").merge(
    df_productsaug21.alias("products_new"),
                    "products.ProductID = products_new.ProductID") \
                    .whenMatchedUpdate(set = { "Price" : "products_new.Price" } ) \
                    .whenNotMatchedInsert(values =
                       {
                        "ProductID": "products_new.ProductID",
                        "Date": "products_new.Date",
                        "Price": "products_new.Price"
                       }
                     ).execute()

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 3).load("hdfs:///delta_lake/products")
df.show()

In [ ]:
spark.table("products").show()

In [ ]:
deltaTable.update("ProductID = '230'", { "Price": "'33.67'" } )
deltaTable.update("ProductID = '210'", { "Price": "'56.00'" } )
deltaTable.update("ProductID = '250'", { "Price": "'99.76'" } )
deltaTable.update("ProductID = '220'", { "Price": "'44.56'" } )
deltaTable.update("ProductID = '240'", { "Price": "'100.82'" } )
deltaTable.update("ProductID = '200'", { "Price": "'35.5'" } )
deltaTable.update("ProductID = '260'", { "Price": "'64.55'" } )
deltaTable.update("ProductID = '280'", { "Price": "'54.78'" } )
deltaTable.update("ProductID = '270'", { "Price": "'106.32'" } )